# API Exercises

## Part A

# 1.Access the Cat Breeds API and download data from all pages


In [ ]:
import requests
import json
from time import sleep
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [ ]:
link = 'https://catfact.ninja/breeds?page={}'
cat_data = []
for i in range(1,5):
    #link.format(i) # here it was blocking and reading looped the same page
    response = requests.get(link.format(i))
    x = json.loads(response.content)
    cat_data.extend(x['data'])


In [ ]:
cat_data
df = pd.DataFrame.from_dict(cat_data)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.info()

In [ ]:
df.drop_duplicates(inplace=True)
df.info()

# 2.Which country has the highest number of cat breeds?


In [ ]:
#df.groupby('country').count().sort_values('breed', ascending=False)#.head(1)['breed']

In [ ]:
#With double bracket at the end returns the result as dataframe !!!!

df.groupby('country').count().sort_values('breed', ascending=False).head(1)[['breed']]#.index[0]

In [ ]:
#reseting index to get it back to columns 
#y.reset_index().set_index('country').index[0]

# 3.What is the percentage of Hairless breeds?

In [ ]:
df.groupby('coat').count().sort_values('breed', ascending=False)['breed'].sum()

In [ ]:
df.groupby('coat').count().sort_values('breed', ascending=False)['breed']['Hairless']

In [ ]:
a = df.groupby('coat').count().sort_values('breed', ascending=False)['breed'].sum()
b = df.groupby('coat').count().sort_values('breed', ascending=False)['breed']['Hairless']
print(f'The percentage of hairless breeds is :{b/a*100}%')

# Part B

# Create an account at https://api-ninjas.com/
1.Enhance the previous dataset with additional cat data! Use the API of cats to get the data and then merge them into one dataframe

In [ ]:
with open('api-ninjas.json', 'r') as fh:
    credentials = json.load(fh)
    
api_key = credentials['key']

In [ ]:
name = 'abyssinian'
def get_cat(name):
    api_url = 'https://api.api-ninjas.com/v1/cats?name={}'.format(name)
    response = requests.get(api_url, headers={'X-Api-Key': api_key})
    if response.status_code == requests.codes.ok:
        return json.loads(response.content)
    else:
        print("Error:", response.status_code, response.text)

In [ ]:
#get_cat(name)

In [ ]:
breed_list = df['breed'].to_list()

In [ ]:
cats_char_list = []
for breed in tqdm(breed_list):
#     print(breed)
    cat_info = get_cat(breed.lower())
    if cat_info:
        x = cat_info[0]
        x.update({'breed':breed})
        cats_char_list.append(x)
    else:
        print(breed)

In [ ]:
df_cats2 = pd.DataFrame(cats_char_list)
#df_cats2.head()

In [ ]:
df_cats_merged = df.merge(df_cats2, on='breed')
#df_cats_merged

In [ ]:
df_cats_merged.dropna(inplace = True)

# 2.In which country do you have the heaviest cats?

In [ ]:
df_cats_merged['avg_weight'] = (df_cats_merged['min_weight'] + df_cats_merged['max_weight'])/2
df_cats_merged.groupby('country')['avg_weight'].mean().sort_values()

# 3.Which cats are the most friendly and playful (find the top 5)? What is their life expectancy?


In [ ]:
df_cats_merged['avg_life'] = (df_cats_merged['min_life_expectancy'] + df_cats_merged['max_life_expectancy'])/2
df_cats_merged[(df_cats_merged['family_friendly'] == 5.0) & (df_cats_merged['playfulness'] == 5.0)]

# Part C (Bonus)

# Create an account at Giant Bomb. Get your API key from the API page.

Now use the following link in your Python script to get data from the API:
'https://www.giantbomb.com/api/games/?filter=platforms%3A35&field_list=id%2Cname%2Coriginal_game_rating%2Coriginal_release_date&sort=name%3Adesc&limit=10&offset=0&api_key=___your___api___key___&format=json'

There is an offset in the link. You have to change the offset repeatedly to get all the results from the API.

To do so you can use the following code using the requests library.
import requests
import json
link = your_link_

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

response = requests.get(link, headers=headers)
json.loads(response.content)
Gather all the data and create a pandas Dataframe.


In [ ]:
with open('api-giantbomb.json', 'r') as fh:
    credentials = json.load(fh)

In [ ]:
link = 'https://www.giantbomb.com/api/games/?filter=platforms%3A35&field_list=id%2Cname%2Coriginal_game_rating%2Coriginal_release_date&sort=name%3Adesc&limit=10&offset=0&api_key={}&format=json'


headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

In [ ]:
response = requests.get(link.format(credentials['key']), headers=headers)
response_json = json.loads(response.content)

In [ ]:
response_json.keys()

In [ ]:
response_json['results']

In [ ]:
link = 'https://www.giantbomb.com/api/games/?filter=platforms%3A35&field_list=id%2Cname%2Coriginal_game_rating%2Coriginal_release_date&sort=name%3Adesc&limit=10&offset={}&api_key={}&format=json'

In [ ]:
results = []
api_key = credentials['key']

for offset in tqdm(range(0,1725,10)):
    response = requests.get(link.format(offset, api_key), headers=headers)
    response_json = json.loads(response.content)
    results.extend(response_json['results'])
#len(results)

In [ ]:
df_gb = pd.DataFrame(results)
df_gb

In [ ]:
df_normalisze = pd.json_normalize(results , max_level=1)
df_normalisze